In [1]:
# Building a CNN with Tensorflow Part 1: Obtaining data
# Download MNIST dataset, Examine images, Examine Labels

In [2]:
import tensorflow as tf
import matplotlib.pyplot as plt
mnist = tf.keras.datasets.mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()
# Part 2a Layers Convolution, flatten layer, dense layer
from tensorflow.keras.layers import Conv2D, Flatten, Dense
from tensorflow.keras import Model
# Part 5 Formatting Data
x_train, x_test = x_train/255.0, x_test/255.0
x_train = x_train[..., tf.newaxis]
x_test = x_test[..., tf.newaxis]

# Shuffle Data
train_data = tf.data.Dataset.from_tensor_slices((x_train, y_train)).shuffle(10000).batch(32)
test_data = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(32)
# Conv2D(filters, kernel_size, activation)
# Flatten()
# Dense(neurons, activation)
# Part 2b Layers practical (build model)
class MNISTModel(Model):
    def __init__(self):
        super(MNISTModel, self).__init__()
        self.conv1 = Conv2D(32, 3, activation='relu')
        self.flatten = Flatten()
        self.dense1 = Dense(128, activation='relu')
        self.dense2 = Dense(10, activation='softmax')
        
    def call(self, x):
        x1 = self.conv1(x)
        x2 = self.flatten(x1)
        x3 = self.dense1(x2)
        return self.dense2(x3)
# Part 3 Loss and optimizer

loss_function = tf.keras.losses.SparseCategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam()
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')
test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')
# Part 4 Implement Train and Test steps
model = MNISTModel()
@tf.function
def train_step(inputs, outputs):
    with tf.GradientTape() as tape:
        predictions = model(inputs)
        loss = loss_function(outputs, predictions)
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    train_loss(loss)
    train_accuracy(outputs, predictions)
    

@tf.function
def test_step(inputs, outputs):
    predictions = model(inputs)
    loss = loss_function(outputs, predictions)
    test_loss(loss)
    test_accuracy(outputs, predictions)

# Part 6 Training and evaluation
epochs = 5

for epoch in range(epochs):
    for train_inputs, train_labels in train_data:
        train_step(train_inputs, train_labels)
    for test_inputs, test_labels in test_data:
        test_step(test_inputs, test_labels)
        
    template = 'Epochs: {}, Train Loss: {}, Train accuracy: {}, Test loss: {} Test accuracy: {}'
    print(template.format(
    epoch + 1,
    train_loss.result(),
    train_accuracy.result(),
    test_loss.result(),
    test_accuracy.result(),
    ))
    train_loss.reset_states()
    train_accuracy.reset_states()
    test_loss.reset_states()
    train_accuracy.reset_states()

Epochs: 1, Train Loss: 0.13612425327301025, Train accuracy: 0.9591666460037231, Test loss: 0.0638522356748581 Test accuracy: 0.9793000221252441
Epochs: 2, Train Loss: 0.04241260886192322, Train accuracy: 0.9865999817848206, Test loss: 0.05642567202448845 Test accuracy: 0.9807999730110168
Epochs: 3, Train Loss: 0.021953964605927467, Train accuracy: 0.9927833080291748, Test loss: 0.052726391702890396 Test accuracy: 0.9817666411399841
Epochs: 4, Train Loss: 0.013241177424788475, Train accuracy: 0.9957166910171509, Test loss: 0.061852652579545975 Test accuracy: 0.9819750189781189
Epochs: 5, Train Loss: 0.009797525592148304, Train accuracy: 0.9965833425521851, Test loss: 0.05689597129821777 Test accuracy: 0.9826200008392334
